### ! - daje odwolanie do czegos w systemie, w chmurze trzeba kliknac pip, bo w chmurze na kilka godzin sie instaluje

## Instalacja pulpa

In [ ]:
!pip install pulp 

     |████████████████████████████████| 40.6MB 106kB/s 


##Importowanie pulpa

In [ ]:
import pulp
print(pulp.__version__)

2.3.1


##Instalacja Sudo

In [ ]:
!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 14 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libco

### Celem tego ćwiczenia było zapoznanie się z gotowym problemem liniowym i dodanie nowych ograniczeń w postaci wymogu chemicznego i dla witaminy B.
### Najpierw trzeba zaimportowac pulp, aby kod realizujący problem liniowy się skompilował.

In [ ]:
from pulp import *

## Tworzenie zmiennej „prob”, aby zawierała dane dotyczące problemu liniowego, który zostanie wykonany

In [ ]:
prob = LpProblem("The Whiskas Problem",LpMinimize) ## LpMinimize służy do określenia wartości minimalnej, jakby było Maximize to do maksymalnej

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Dwie zmienne decyzyjne są tworzone z dolnym limitem zerowym

In [ ]:
x1=LpVariable("ChickenPercent",0.3,None,LpInteger)
x2=LpVariable("BeefPercent",2.6)
x3=LpVariable("Procent_Chemii",1)
x4=LpVariable("Procent_Mutton",2.6)
x5=LpVariable("Sardynki",8.9)

##Zdefiniowanie funkcji zysku

In [ ]:
prob += 0.013*x1 + 0.008*x2, "Total Cost of Ingredients per can"

## W tym miejscu zostało wprowadzonych początkowo 5 ograniczeń, jednak następnie należało dodać kolejne 2 w celu ukazania wymogu chemicznego i dla witaminy B.

In [ ]:
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 <= 0.4, "SaltRequirement"
prob += 1000*x1 + 0.0076*x2 + 200000*x3 <= 1, "wymogchemiczny"
prob += 0.002*x1 + 0.0002*x2 +0.005*x3 + 0.005*x4 + 0.079*x5 <=1, "witaminaB"

## Dane problemu są zapisywane w pliku .lp

In [ ]:
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent, Procent_Chemii, Procent_Mutton, Sardynki]

## Problem został rozwiązany przy użyciu Solvera wybranego przez PuLP

In [ ]:
prob.solve()

-1

## Stan rozwiązania jest pokazywany poniżej po skompilowaniu

In [ ]:
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Infeasible
BeefPercent = 66.666667
ChickenPercent = 33.333333
Procent_Chemii = 1.0
Procent_Mutton = 2.6
Sardynki = 8.9


## Zoptymalizowana wartość funkcji zysku jest drukowana na ekranie

In [ ]:
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.966666665


## Jak widać program działa prawidłowo. Po zmianie mnożników w ograniczeniach wartości się zmieniają liniowo.

## Ten kod jest wersją rozbudowaną tego powyższego

In [ ]:

"""
The Full Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeler functions
from pulp import *

# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
